In [11]:
from selenium import webdriver
import pandas as pd
import numpy as np

In [12]:
import requests
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
page = requests.get(url,headers=headers)
page

<Response [200]>

In [13]:
nifty_margin = 500
bank_nifty_margin = 1000
stock_margin = 10

In [14]:
col_list = ['strikePrice','expiryDate','openInterest']
# data_format = ['Date','Stock/Index','Expiry','Right','Open Price','Close Price','Current OI','Previous Minute OI',
#                'OI Change','% Up/Down from Last Minute OI']

In [15]:
selected_margin = nifty_margin

In [16]:
import json
res = json.loads(page.text)['records']['data']

In [17]:
underlying_value = int(res[0]['PE']['underlyingValue'])
type(underlying_value)

int

In [18]:
data = [d.get('CE') for d in res]
data_cleaned = list(filter(None, data))
df_call = pd.DataFrame(data_cleaned)

data = [d.get('PE') for d in res]
data_cleaned = list(filter(None, data))
df_put = pd.DataFrame(data_cleaned)

df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
df_call['expiryDate'] = pd.to_datetime(df_call['expiryDate'])


df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)
df_call.sort_values(by=['expiryDate','strikePrice'],inplace=True)

df_put = df_put[col_list]
df_call = df_call[col_list]

In [19]:
df_put['strikePrice'] = df_put['strikePrice'].map(int)
df_call['strikePrice'] = df_call['strikePrice'].map(int)

In [31]:
# df_call
res = json.loads(page.text)
print(res)

{'records': {'expiryDates': ['27-May-2021', '03-Jun-2021', '10-Jun-2021', '17-Jun-2021', '24-Jun-2021', '01-Jul-2021', '08-Jul-2021', '15-Jul-2021', '22-Jul-2021', '29-Jul-2021', '30-Sep-2021', '30-Dec-2021', '31-Mar-2022', '30-Jun-2022', '29-Dec-2022', '29-Jun-2023', '28-Dec-2023', '26-Jun-2025'], 'data': [{'strikePrice': 6000, 'expiryDate': '24-Jun-2021', 'PE': {'strikePrice': 6000, 'expiryDate': '24-Jun-2021', 'underlying': 'NIFTY', 'identifier': 'OPTIDXNIFTY24-06-2021PE6000.00', 'openInterest': 18, 'changeinOpenInterest': 0, 'pchangeinOpenInterest': 0, 'totalTradedVolume': 0, 'impliedVolatility': 0, 'lastPrice': 2.5, 'change': 0, 'pChange': 0, 'totalBuyQuantity': 5025, 'totalSellQuantity': 450, 'bidQty': 75, 'bidprice': 1.25, 'askQty': 75, 'askPrice': 3, 'underlyingValue': 15175.3}}, {'strikePrice': 7500, 'expiryDate': '29-Dec-2022', 'PE': {'strikePrice': 7500, 'expiryDate': '29-Dec-2022', 'underlying': 'NIFTY', 'identifier': 'OPTIDXNIFTY29-12-2022PE7500.00', 'openInterest': 14, 'c

In [20]:
# min, max = ((100-percentage)*0.01)*(df['underlyingValue'][0]), ((100+percentage)*0.01)*(df['underlyingValue'][0])
# df_percent = df.loc[(df['strikePrice']>=min) & (df['strikePrice']<=max)]
# df_percent

In [21]:
df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]
df_call = df_call[(df_call['strikePrice'] >= underlying_value-selected_margin) & (df_call['strikePrice']<=underlying_value+selected_margin)]
df_put['RIGHT'] = 'PUT'
df_call['RIGHT'] = 'CALL'

In [22]:
max_sp = df_call.strikePrice.max
min_sp = df_call.strikePrice.min

In [23]:
sp_range = df_put.strikePrice.unique().tolist()

In [24]:
for i in sp_range:
    temp_df_put = df_put[df_put['strikePrice']==i]
    temp_df_call = df_call[df_call['strikePrice']==i]

In [25]:
temp1 = df_call
temp2 = df_put
# Ispe bharosa mat karna
temp1.merge(temp2,left_index=True,right_index=True,on='strikePrice')

,strikePrice,expiryDate_x,openInterest_x,RIGHT_x,expiryDate_y,openInterest_y,RIGHT_y
426,14800,2021-05-27,22670,CALL,2021-05-27,31764,PUT
436,14850,2021-05-27,1022,CALL,2021-05-27,7128,PUT
446,14900,2021-05-27,13897,CALL,2021-06-03,2771,PUT
457,14950,2021-05-27,2153,CALL,2021-05-27,7821,PUT
465,15000,2021-05-27,35065,CALL,2021-07-22,0,PUT
...,...,...,...,...,...,...,...
570,15500,2021-09-30,3,CALL,2021-07-22,0,PUT
467,15000,2021-12-30,5037,CALL,2021-05-27,39429,PUT
562,15500,2021-12-30,329,CALL,2021-07-08,0,PUT
468,15000,2022-03-31,0,CALL,2021-06-17,0,PUT


# LEFT

### Sample code

In [26]:
import pandas as pd

df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd'], 'D': ['a', 'bb', 'c', 'dd']})

df2 = pd.DataFrame({'Data': [1, 2, 3, 4],  'D': ['a', 'bb', 'c', 'dd']})

c = pd.concat([df1, df2], axis=1)
c

,Data,D,Data,D
0,a,a,1,a
1,b,bb,2,bb
2,c,c,3,c
3,d,dd,4,dd


### Sample code

In [27]:
import pandas as pd
import xlsxwriter

df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd']})

df2 = pd.DataFrame({'Data': [1, 2, 3, 4]})

df3 = pd.DataFrame({'Data': [1.1, 1.2, 1.3, 1.4]})

writer = pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter')

df1.to_excel(writer, sheet_name='Sheet_1')

df2.to_excel(writer, sheet_name='Sheet_2')

df3.to_excel(writer, sheet_name='Sheet_3')

writer.save()